In [1]:
# 初始化
%load_ext autoreload
%autoreload 2
import sys
import os
os.chdir('E:\GitHub\QA-abstract-and-reasoning')
sys.path.append('E:\GitHub\QA-abstract-and-reasoning')

In [5]:
import tensorflow as tf
from pgn.layers import BahdanauAttention, Encoder, Pointer
from pgn.batcher import batcher
from pgn.model import PGN
from utils.saveLoader import load_embedding_matrix
from utils.saveLoader import Vocab
from utils.config import VOCAB_PAD
from utils.config_gpu import config_gpu
config_gpu()

1 Physical GPUs, 1 Logical GPUs


In [6]:
%run utils/params.py
# 产生输入数据
vocab = Vocab(VOCAB_PAD)
ds =iter(batcher(vocab, params))
enc_data, dec_data = next(ds)

In [7]:
config_gpu()

print("Building the model ...")

model = PGN(params)

print("Creating the vocab ...")
vocab = Vocab(params["vocab_path"], params["vocab_size"])
# ds = batcher(vocab, params)

print("Creating the checkpoint manager")
checkpoint = tf.train.Checkpoint(Seq2Seq=model)

checkpoint_manager = tf.train.CheckpointManager(checkpoint, PGN_CKPT, max_to_keep=5)

# checkpoint_manager = tf.train.CheckpointManager(checkpoint, TEMP_CKPT, max_to_keep=5)
# temp_ckpt = os.path.join(TEMP_CKPT, "ckpt-5")
# checkpoint.restore(temp_ckpt)

checkpoint.restore(checkpoint_manager.latest_checkpoint)
if checkpoint_manager.latest_checkpoint:
    print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")
print("Model restored")

1 Physical GPUs, 1 Logical GPUs
Building the model ...
Creating the vocab ...
Creating the checkpoint manager
Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\pgn_checkpoints\ckpt-1
Model restored


In [9]:
from pgn.test_helper import decode_one_step

In [11]:
batch_data = enc_data["enc_input"]
batch_size = params["batch_size"]

In [14]:
predicts = [''] * batch_size

In [16]:
inputs = batch_data

In [17]:
enc_output, enc_hidden = model.encoder(inputs)
dec_hidden = enc_hidden